In [6]:
import numpy as np
import pandas as pd
import warnings
import pulp
warnings.filterwarnings('ignore')

In [39]:
outbound_data = pd.read_excel("Outbound.xlsx")
demand_data = pd.read_excel("Demand Forecast.xlsx")
unit_data = pd.read_excel('Product Data per State.xlsx')
handling_out_data = pd.read_excel('Warehousing.xlsx')
storage_data = pd.read_excel('Storage.xlsx')

In [31]:
# What I did: removed the State column, remove the description of tariff and small shipments
outbound_data.index = outbound_data.State
demand_data.index = demand_data.state
outbound_data = outbound_data.drop(['State','Small shipment'], axis=1)

In [32]:
# As-Is situation

dc_allocation = {}
# then check everything! (also ask why DC is not in the list)
for state in outbound_data.index:
    best_dc = (outbound_data.loc[state]).idxmin()
    
    if best_dc not in dc_allocation:
        dc_allocation[best_dc] = state
    elif isinstance(dc_allocation[best_dc], list):
        dc_allocation[best_dc].append(state)
    else:
        dc_allocation[best_dc] = [dc_allocation[best_dc], state]
        
dc_allocation['CA'] = ['CA']
print(dc_allocation)

def outbound_costs(dc_allocation):
    dc_product_demand = {}
    for dc in dc_allocation:
        dc_product_demand[dc] = {}  # Create a dictionary for each DC
        for product in ['blender', 'swing', 'chair', 'scooter', 'skiprope']:
            total_demand_per_dc = 0
            for state in dc_allocation[dc]:
                total_demand_per_dc += (demand_data.loc[state, product])
            dc_product_demand[dc][product] = total_demand_per_dc  # Store demand (in boxes) for each product based on DC

    # Calculate the outbound costs
    shipping_costs = []
    handling_out_costs_per_dc = []
    product_units = unit_data.columns.difference(['state'])
    handling_out_data.index = handling_out_data.DC
    unit_data.index = unit_data.state

    for state in demand_data.index:
        for dc, states in dc_allocation.items():
            if state in states:
                shipping_costs.append((demand_data['total_weight'][state] * outbound_data.loc[state, dc]))
                # handling_out_costs.append(demand_data['blender_boxes'] * handling_out_data.loc[dc, ])

    for dc, states in dc_allocation.items():
        out_costs_per_state = 0
        for state in unit_data.index:
            if state in states:
                for product in product_units:
                    out_costs_per_state += unit_data[product][state] * handling_out_data.loc[dc, product]
                
        handling_out_costs_per_dc.append(out_costs_per_state)

    shipping_costs = sum(shipping_costs)
    handling_out_costs = sum(handling_out_costs_per_dc)
    opening_costs = len(dc_allocation)*1000000 # Keep a DC open need $1M per year.
    total_outbound_costs = np.round(shipping_costs + handling_out_costs + opening_costs, 1)

    print('Total Shipping Costs (from DC to Customers):', shipping_costs)
    print('Total Handling Out Costs:', handling_out_costs)
    print('Total costs for keeping 7 DCs open:', opening_costs)
    print('Total Outbound Costs:', total_outbound_costs)

{'WA': ['AK', 'ID', 'MT', 'OR', 'WA'], 'TN': ['AL', 'FL', 'GA', 'KY', 'MS', 'NC', 'OH', 'SC', 'VA', 'WV'], 'TX': ['AR', 'LA', 'NM', 'OK', 'TN', 'TX'], 'UT': ['AZ', 'CO', 'NV', 'UT', 'WY'], 'CA': ['CA'], 'NY': ['CT', 'DE', 'MA', 'ME', 'NH', 'NJ', 'NY', 'RI', 'VT'], 'PA': ['HI', 'MD', 'PA', 'DC'], 'KS': ['IA', 'KS', 'MO', 'NE'], 'IL': ['IL', 'MI', 'MN'], 'ND': ['IN', 'ND', 'SD', 'WI']}
Total Shipping Costs (from DC to Customers): 911216.61240381
Total Handling Out Costs: 1868259.8699999999
Total costs for keeping 7 DCs open: 10000000
Total Outbound Costs: 12779476.5


In [34]:
as_is_dc = {
    "NY": ['CT', 'DC', 'DE', 'MA', 'MD', 'ME', 'NH', 'NJ', 'NY', 'PA', 'RI', 'VT'],
    "ND": ['MN', 'MT', 'ND', 'SD', 'WY'],
    "IL": ['HI', 'IA', 'IL', 'IN', 'KS', 'KY', 'MI', 'MO', 'NE', 'OH', 'VA', 'WI', 'WV'],
    "TN": ['AL', 'FL', 'GA', 'NC', 'SC', 'TN'],
    "WA": ['AK', 'ID', 'OR', 'WA'],
    "TX": ['AR', 'AZ', 'LA', 'MS', 'NM', 'OK', 'TX'],
    "UT": ['CA', 'CO', 'NV', 'UT']
}

outbound_costs(as_is_dc)

Total Shipping Costs (from DC to Customers): 1135481.33914202
Total Handling Out Costs: 1845542.21
Total costs for keeping 7 DCs open: 7000000
Total Outbound Costs: 9981023.5


In [38]:
# Determine To-Be situation

# Know the best DC allocation based on distance only
best_dc_allocation = {}
for state in outbound_data.index:
    best_dc = (outbound_data.loc[state]).idxmin()
    
    if best_dc not in best_dc_allocation:
        best_dc_allocation[best_dc] = [state]
    elif isinstance(best_dc_allocation[best_dc], list):
        best_dc_allocation[best_dc].append(state)
    else:
        best_dc_allocation[best_dc] = [best_dc_allocation[best_dc], state]
        
# print(best_dc_allocation)
outbound_costs(best_dc_allocation)

Total Shipping Costs (from DC to Customers): 911216.61240381
Total Handling Out Costs: 1868259.8699999999
Total costs for keeping 7 DCs open: 10000000
Total Outbound Costs: 12779476.5


In [ ]:

# Create the optimization problem
prob = pulp.LpProblem("UFLP_with_Current_Status", pulp.LpMinimize)

# Decision variables
y = pulp.LpVariable.dicts("OpenFacility", range(num_dc), cat='Binary')
x = pulp.LpVariable.dicts("ServeCustomer", (range(num_states), range(num_dc)), cat='Binary')

# Objective function
# fc = pulp.lpSum(fixed_costs[j] * y[j] for j in range(num_dc))
# vc = pulp.lpSum(variable_costs[i][j] * x[i][j] for i in range(num_states) for j in range(num_dc))
# prob +=  fc + vc
        

# Constraints
for i in range(num_states):
    prob += pulp.lpSum(x[i][j] for j in range(num_dc)) == 1  # Each customer is assigned to exactly one facility

for i in range(num_states):
    for j in range(num_dc):
        prob += x[i][j] <= y[j]  # Customers can only be assigned to open facilities

# Solve the problem
prob.solve()

# Print results
print("Status:", pulp.LpStatus[prob.status])

print("Facilities to open:")
for j in range(num_dc):
    if y[j].varValue == 1:
        print(f"Facility {j} should be opened.")

print("Customer assignments:")
for i in range(num_states):
    for j in range(num_dc):
        if x[i][j].varValue == 1:
            print(f"Customer {i} is served by Facility {j}.")


In [14]:
# Determine To-Be situation
best_dc_allocation = {}
# then check everything! (also ask why DC is not in the list)
for state in outbound_data.index:
    best_dc = (outbound_data.loc[state]).idxmin()
    
    if best_dc not in best_dc_allocation:
        best_dc_allocation[best_dc] = state
    elif isinstance(best_dc_allocation[best_dc], list):
        best_dc_allocation[best_dc].append(state)
    else:
        best_dc_allocation[best_dc] = [best_dc_allocation[best_dc], state]
        
outbound_costs(best_dc_allocation)


KeyError: 'C'